# Worksheet 08

Name: Sinforiano Terada 
UID: U80138659

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.423976666576214, 4.750607593765246, 3.7774956891358795, 4.744868572674481, 5.341023201986534, 5.7087868173208145, 4.816756819173796, 5.012094482602794, 5.216634993231939, 4.409066069878011]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[6.692668920181799, 7.419539618791397, 7.422792953709893, 7.433501228512353, 6.023817121447956, 9.852732215480525, 8.165686222294246, 10.40646102680033, 7.974284392517058, 6.989983245332715]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[4.409066069878011, 6.989983245332715, 7.974284392517058, 10.40646102680033, 8.165686222294246, 5.216634993231939, 9.852732215480525, 6.023817121447956, 7.433501228512353, 7.422792953709893]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The parameters of the GMM are the mean and standard deviation for the two distributions as well as the probability that they landed in data.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] != 0, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) /(len(s2) + len(s1)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[6.989983245332715, 7.974284392517058, 10.40646102680033, 8.165686222294246, 9.852732215480525, 7.433501228512353, 7.422792953709893]
[4.409066069878011, 5.216634993231939, 6.023817121447956]
P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 8.32077732637816,  mean_2 = 5.2165060615193015
var_1 = 1.4579600825398544,  var_2 = 0.43457016806940035


I think that they're pretty similar since all we're doing is computing the mean, variance, and probability of points coming from the individual data sets. I think our results for the next part will be different though because this isn't necessarily the correct method

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.409066069878011
probability of observing that point if it came from cluster 0 =  0.007482193341313717
probability of observing that point if it came from cluster 1 =  0.16338277314381863
point =  6.989983245332715
probability of observing that point if it came from cluster 0 =  0.18040359429020447
probability of observing that point if it came from cluster 1 =  0.00022201031940165023
point =  7.974284392517058
probability of observing that point if it came from cluster 0 =  0.2660111720568522
probability of observing that point if it came from cluster 1 =  1.6518979877380012e-09
point =  10.40646102680033
probability of observing that point if it came from cluster 0 =  0.09835101987192806
probability of observing that point if it came from cluster 1 =  9.803709965745126e-32
point =  8.165686222294246
probability of observing that point if it came from cluster 0 =  0.2720866622285645
probability of observing that point if it came from cluster 1 =  9.16211606565729e-11
point =

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [9]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([(p * (x - mean[0])**2) for p, x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([(p * (x - mean[1])**2) for p, x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 8.086322628638822,  mean_2 = 5.06509018298242
var_1 = 1.9113423355909416,  var_2 = 0.3659155291730864


The P and the means are quite similar, but there is a more notable difference in the variances.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [10]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  4.409066069878011
probability of observing that point if it came from cluster 0 =  0.03279546186836836
probability of observing that point if it came from cluster 1 =  0.21855788751512423
point =  6.989983245332715
probability of observing that point if it came from cluster 0 =  0.17706279644869374
probability of observing that point if it came from cluster 1 =  1.067772678172146e-06
point =  7.974284392517058
probability of observing that point if it came from cluster 0 =  0.2083653329677998
probability of observing that point if it came from cluster 1 =  2.0498246189678618e-14
point =  10.40646102680033
probability of observing that point if it came from cluster 0 =  0.0999090024286014
probability of observing that point if it came from cluster 1 =  5.857046792879087e-47
point =  8.165686222294246
probability of observing that point if it came from cluster 0 =  0.20854376027467833
probability of observing that point if it came from cluster 1 =  2.793860788274018e-16
point = 

I think that these values make a lot more sense. Some of the probabilities in the KMeans section didn't really make sense, but I think that the probabilities shown here are closer to what I would estimate with the eye test.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [8]:
assignments = [[], []]
for x in zip(data, prob_s0_x):
    p = x[1]
    d = x[0]
    if random.random() <= p:
        assignments[0].append(d)
    else:
        assignments[1].append(d)
print(f"points in cluster 0 : {assignments[0]}\npoints in cluster 1 : {assignments[1]}")

points in cluster 0 : [6.989983245332715, 7.974284392517058, 10.40646102680033, 8.165686222294246, 9.852732215480525, 7.433501228512353, 7.422792953709893]
points in cluster 1 : [4.409066069878011, 5.216634993231939, 6.023817121447956]
